# Importador de Dados ANS para PostgreSQL

Este notebook permite importar diferentes datasets da ANS para um banco de dados PostgreSQL.
Você pode configurar o diretório dos arquivos, a tabela de destino e outras opções conforme necessário.

## Instalação das Dependências

Primeiro, vamos instalar os pacotes necessários caso não estejam disponíveis.

In [5]:
# Instalação dos pacotes necessários
!pip install pandas psycopg2-binary sqlalchemy ipython-sql


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
import glob
from IPython.display import display, HTML

## Configurações

In [7]:
# Configurações de conexão com o banco de dados
db_config = {
    'host': '146.235.222.230',  
    'port': '5432',
    'database': 'ans',
    'user': 'vcollos',
    'password': 'Essaaqui:#01'
}

## Funções de Conexão e Importação

In [5]:
def create_connection():
    """Cria conexão com o banco de dados PostgreSQL"""
    try:
        # Conexão via psycopg2 para executar comandos SQL diretos
        conn = psycopg2.connect(
            host=db_config['host'],
            port=db_config['port'],
            database=db_config['database'],
            user=db_config['user'],
            password=db_config['password']
        )
        print("Conexão com PostgreSQL estabelecida com sucesso!")
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao PostgreSQL: {e}")
        return None

def create_sqlalchemy_engine():
    """Cria engine SQLAlchemy para operações com pandas"""
    conn_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    return create_engine(conn_string)

def check_table_exists(conn, table_name):
    """Verifica se a tabela já existe no banco de dados"""
    try:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_schema = 'public'
                AND table_name = %s
            );
        """, (table_name,))
        exists = cursor.fetchone()[0]
        return exists
    except Exception as e:
        print(f"Erro ao verificar se a tabela existe: {e}")
        return False

## Funções de Processamento de Dados

In [6]:
def read_csv_file(file_path, sep=';', encoding='utf-8', date_columns=None):
    """Lê um arquivo CSV usando pandas"""
    try:
        # Lendo CSV com o separador especificado
        df = pd.read_csv(file_path, sep=sep, encoding=encoding, low_memory=False)
        file_name = os.path.basename(file_path)
        print(f"Arquivo {file_name} lido com sucesso. {len(df)} registros encontrados.")
        
        # Convertendo colunas de data para o formato correto (se especificadas)
        if date_columns:
            for col in date_columns:
                if col in df.columns:
                    # Converter para datetime, lidar com valores vazios
                    df[col] = pd.to_datetime(df[col], errors='coerce')
        
        return df
    except Exception as e:
        print(f"Erro ao ler arquivo {file_path}: {e}")
        return None

def import_data(df, engine, table_name, if_exists='append'):
    """Importa os dados do DataFrame para a tabela PostgreSQL"""
    try:
        # Importando dados via to_sql do pandas
        df.to_sql(table_name, engine, if_exists=if_exists, index=False, schema='public')
        print(f"Dados importados com sucesso para a tabela '{table_name}'! {len(df)} registros inseridos.")
        return True
    except Exception as e:
        print(f"Erro ao importar dados para a tabela '{table_name}': {e}")
        return False

## Importador Flexível

Esta função permite importar qualquer conjunto de arquivos para qualquer tabela no banco.

In [7]:
def importar_dados(dir_path, file_pattern, table_name, date_columns=None, 
                   exclude_files=None, sep=';', encoding='utf-8', if_exists='append', 
                   create_if_not_exists=True):
    """Função flexível para importar dados de múltiplos arquivos para uma tabela específica"""
    # Estabelecer conexão com o banco
    conn = create_connection()
    if not conn:
        return
    
    # Criar engine SQLAlchemy
    engine = create_sqlalchemy_engine()
    
    # Verificar se a tabela existe
    tabela_existe = check_table_exists(conn, table_name)
    print(f"Tabela '{table_name}' existe: {tabela_existe}")
    
    # Obter todos os arquivos CSV no diretório que seguem o padrão
    csv_files = glob.glob(os.path.join(dir_path, file_pattern))
    
    # Excluir arquivos específicos, se solicitado
    if exclude_files:
        for exclude_file in exclude_files:
            exclude_path = os.path.join(dir_path, exclude_file)
            if exclude_path in csv_files:
                csv_files.remove(exclude_path)
                print(f"Arquivo excluído do processamento: {exclude_file}")
    
    print(f"Encontrados {len(csv_files)} arquivos para processar.")
    
    # Se tabela não existe e create_if_not_exists=True, criar a tabela com base no primeiro arquivo
    if not tabela_existe and create_if_not_exists and len(csv_files) > 0:
        print(f"Criando tabela '{table_name}' com base no primeiro arquivo...")
        sample_file = csv_files[0]
        sample_df = read_csv_file(sample_file, sep=sep, encoding=encoding, date_columns=date_columns)
        if sample_df is not None:
            if create_table(conn, engine, table_name, sample_df):
                tabela_existe = True
    
    # Processar cada arquivo
    total_registros = 0
    for file_path in csv_files:
        print(f"\nProcessando arquivo: {os.path.basename(file_path)}")
        
        # Ler arquivo CSV
        df = read_csv_file(file_path, sep=sep, encoding=encoding, date_columns=date_columns)
        if df is None:
            continue
        
        # Mostrar amostra dos dados
        print("\nAmostra dos dados:")
        display(df.head(3))
        
        # Importar dados
        success = import_data(df, engine, table_name, if_exists=if_exists)
        if success:
            total_registros += len(df)
    
    # Fechar conexão
    conn.close()
    print(f"\nProcesso concluído! Total de {total_registros} registros importados de {len(csv_files)} arquivos para a tabela '{table_name}'.")
    
    return total_registros

In [8]:
def create_table(conn, engine, table_name, sample_df=None):
    """Cria a tabela no banco de dados com base no DataFrame de amostra"""
    try:
        if sample_df is not None:
            # Usar o DataFrame para criar a tabela com tipos adequados
            # Limitamos a 0 registros para apenas criar a estrutura sem inserir dados
            sample_df.head(0).to_sql(table_name, engine, if_exists='fail', index=False)
            print(f"Tabela '{table_name}' criada com sucesso baseada no schema do DataFrame!")
            return True
        else:
            print("Erro: DataFrame de amostra não fornecido para criação da tabela.")
            return False
    except Exception as e:
        print(f"Erro ao criar tabela '{table_name}': {e}")
        return False

## Exemplo de Uso - Importação de Prestadores

Aqui está um exemplo de como usar a função para importar os prestadores não hospitalares:

In [9]:
# Exemplo de importação de prestadores não hospitalares
dir_prestadores = '/Volumes/M1_SSD/DEV/ANS/Arquivos/Cidades de Atuação/'
dir_cidades = '/Volumes/M1_SSD/DEV/ANS/Arquivos/Cidades de Atuação/'
#data_colunas_prestadores = [
#    'DT_VINCULO_OPERADORA_INICIO', 
#    'DT_VINCULO_OPERADORA_FIM', 
#    'DT_ATUALIZACAO'
#]
#excluir_arquivos = ['opera_presta_nao_hospitalares_202502_RR.csv']


importar_dados(
    dir_path=dir_cidades,
    file_pattern='cidades.csv',
    table_name='cidades',
    sep=',',  # Ajuste para usar vírgula como separador
    if_exists='append'  # Use 'replace' para substituir a tabela ou 'fail' para falhar se a tabela existir
)

Conexão com PostgreSQL estabelecida com sucesso!
Tabela 'cidades' existe: True
Encontrados 0 arquivos para processar.

Processo concluído! Total de 0 registros importados de 0 arquivos para a tabela 'cidades'.


0

## Importação de Outros Tipos de Dados

Você pode criar células adicionais para importar outros tipos de dados, como demonstrado abaixo:

In [10]:
# Exemplo para importar outro tipo de dados (modifique conforme necessário)
'''
dir_outros_dados = '/Volumes/M1_SSD/DEV/ANS/Arquivos/outro_diretorio/'
data_colunas_outros = [
    'DATA_INICIO', 
    'DATA_FIM'
]

importar_dados(
    dir_path=dir_outros_dados,
    file_pattern='padrao_arquivo_*.csv',
    table_name='outra_tabela',
    date_columns=data_colunas_outros,
    if_exists='append'
)
'''

"\ndir_outros_dados = '/Volumes/M1_SSD/DEV/ANS/Arquivos/outro_diretorio/'\ndata_colunas_outros = [\n    'DATA_INICIO', \n    'DATA_FIM'\n]\n\nimportar_dados(\n    dir_path=dir_outros_dados,\n    file_pattern='padrao_arquivo_*.csv',\n    table_name='outra_tabela',\n    date_columns=data_colunas_outros,\n    if_exists='append'\n)\n"

## Importação de Dados Hospitalares

In [11]:
# Exemplo para importar dados de prestadores hospitalares
'''
dir_hospitalares = '/Volumes/M1_SSD/DEV/ANS/Arquivos/prestadores/arquivos/operadoras_e_prestadores_hospitalares/'
data_colunas_hospitalares = [
    'DT_VINCULO_OPERADORA_INICIO', 
    'DT_VINCULO_OPERADORA_FIM', 
    'DT_ATUALIZACAO'
]

importar_dados(
    dir_path=dir_hospitalares,
    file_pattern='opera_presta_hospitalares_*.csv',
    table_name='prestadores_hospitalares',
    date_columns=data_colunas_hospitalares,
    if_exists='append'
)
'''

"\ndir_hospitalares = '/Volumes/M1_SSD/DEV/ANS/Arquivos/prestadores/arquivos/operadoras_e_prestadores_hospitalares/'\ndata_colunas_hospitalares = [\n    'DT_VINCULO_OPERADORA_INICIO', \n    'DT_VINCULO_OPERADORA_FIM', \n    'DT_ATUALIZACAO'\n]\n\nimportar_dados(\n    dir_path=dir_hospitalares,\n    file_pattern='opera_presta_hospitalares_*.csv',\n    table_name='prestadores_hospitalares',\n    date_columns=data_colunas_hospitalares,\n    if_exists='append'\n)\n"

## Consulta para Verificar os Dados Importados

In [12]:
def verificar_tabela(table_name, limit=10):
    """Função para verificar os dados de uma tabela após a importação"""
    conn = create_connection()
    if not conn:
        return
    
    engine = create_sqlalchemy_engine()
    
    try:
        # Contar registros
        count_query = f"SELECT COUNT(*) FROM {table_name}"
        count = pd.read_sql(count_query, engine).iloc[0, 0]
        print(f"Tabela '{table_name}' contém {count} registros.")
        
        # Mostrar amostra
        sample_query = f"SELECT * FROM {table_name} LIMIT {limit}"
        sample = pd.read_sql(sample_query, engine)
        display(sample)
    except Exception as e:
        print(f"Erro ao verificar a tabela: {e}")
    finally:
        conn.close()

# Verificar a tabela de prestadores após a importação
# verificar_tabela('prestadores')

In [13]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Cidades de Atuação/cidades.csv')

# Get all column names
columns = df.columns.tolist()

# Remove QT_HAB_MUNICIPIO from the columns to be quoted
columns.remove('QT_HAB_MUNICIPIO')

# Apply quotes to all string columns except QT_HAB_MUNICIPIO
for column in columns:
    df[column] = '"' + df[column].astype(str) + '"'

# Save the modified dataframe back to CSV
df.to_csv('/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Cidades de Atuação/cidades.csv', index=False)


In [17]:
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [19]:
import pandas as pd
import requests
import time
import json
from datetime import datetime
import os
from tqdm import tqdm  # Versão padrão do tqdm em vez da versão notebook

# URL base da API
base_url = 'https://www.ans.gov.br/operadoras-entity/v1/operadoras'

# Diretório para salvar os dados
output_dir = "/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras"
# Criar o diretório se não existir
os.makedirs(output_dir, exist_ok=True)

# Timestamp para os arquivos
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Função para coletar todas as operadoras (lista básica)
def get_all_operadoras():
    print(f"Iniciando coleta da lista de operadoras em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}...")
    
    # Parâmetros iniciais
    params = {
        'size': 100,  # Máximo de registros por página
        'sort': 'registro_ans',
        'page': 0
    }
    
    all_records = []
    page = 0
    more_pages = True
    total_records = 0
    
    while more_pages:
        params['page'] = page
        print(f"Coletando página {page+1}...")
        
        try:
            response = requests.get(base_url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                
                # Verifica o formato da resposta
                if isinstance(data, dict) and 'content' in data:
                    records = data['content']
                    total_pages = data.get('totalPages', -1)
                    total_elements = data.get('totalElements', -1)
                    print(f"Informações de paginação: Página {page+1} de {total_pages}, Total de elementos: {total_elements}")
                elif isinstance(data, list):
                    records = data
                else:
                    print(f"Estrutura de dados inesperada. Verificando primeira parte...")
                    print(str(data)[:500])
                    break
                
                num_records = len(records)
                
                if num_records == 0:
                    more_pages = False
                    print("Nenhum registro encontrado na página. Finalizando coleta.")
                    break
                    
                all_records.extend(records)
                total_records += num_records
                
                print(f"Coletados {num_records} registros da página {page+1}. Total até agora: {total_records}")
                
                # Se retornou menos registros que o tamanho da página, é a última página
                if num_records < params['size']:
                    more_pages = False
                    print("Última página alcançada.")
                    break
                    
                page += 1
                time.sleep(1)  # Pausa para não sobrecarregar a API
                
            else:
                print(f"Erro na requisição. Status code: {response.status_code}")
                print(f"Resposta: {response.text}")
                more_pages = False
                break
                
        except Exception as e:
            print(f"Erro ao processar a página {page+1}: {str(e)}")
            more_pages = False
            break
    
    print(f"Coleta da lista básica finalizada. Total de {len(all_records)} operadoras.")
    return all_records

# Função para obter detalhes de uma operadora específica
def get_operadora_details(registro_ans):
    url = f"{base_url}/{registro_ans}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Erro ao obter detalhes da operadora {registro_ans}. Status: {response.status_code}")
            return None
    except Exception as e:
        print(f"Erro na requisição para operadora {registro_ans}: {str(e)}")
        return None

# Função principal
def main():
    # 1. Obter a lista de todas as operadoras
    operadoras_lista = get_all_operadoras()
    
    if not operadoras_lista:
        print("Nenhuma operadora encontrada.")
        return
    
    # Salvar a lista básica (caso a próxima etapa falhe)
    basic_df = pd.DataFrame(operadoras_lista)
    basic_file = f"{output_dir}/ans_operadoras_lista_basica_{timestamp}.csv"
    basic_df.to_csv(basic_file, index=False, encoding='utf-8')
    print(f"Lista básica salva em: {basic_file}")
    
    # Extrair os registros ANS para buscar detalhes
    registros_ans = []
    for operadora in operadoras_lista:
        if 'registro_ans' in operadora:
            registros_ans.append(operadora['registro_ans'])
    
    print(f"Obtidos {len(registros_ans)} registros ANS para busca detalhada.")
    
    # 2. Obter detalhes de cada operadora
    print(f"Iniciando coleta de detalhes de cada operadora em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}...")
    
    operadoras_detalhadas = []
    
    # Salvar também os JSONs individuais para cada operadora
    json_dir = f"{output_dir}/json_operadoras_{timestamp}"
    os.makedirs(json_dir, exist_ok=True)
    
    # Contador simples para mostrar progresso
    total_registros = len(registros_ans)
    
    # Usar tqdm padrão para mostrar barra de progresso
    for i, registro in enumerate(tqdm(registros_ans, desc="Coletando detalhes")):
        details = get_operadora_details(registro)
        
        if details:
            # Salvar o JSON individual
            with open(f"{json_dir}/{registro}.json", 'w', encoding='utf-8') as f:
                json.dump(details, f, ensure_ascii=False, indent=2)
            
            # Tratar "_links" que é um objeto aninhado
            if '_links' in details:
                # Transformar links em colunas planas
                for link_key, link_value in details['_links'].items():
                    prefix = f"link_{link_key}_"
                    for k, v in link_value.items():
                        details[prefix + k] = v
                # Remover o objeto original
                del details['_links']
            
            operadoras_detalhadas.append(details)
        
        # Log periódico para acompanhar o progresso (a cada 50 operadoras)
        if (i + 1) % 50 == 0:
            print(f"Processados {i + 1}/{total_registros} registros ({(i + 1)/total_registros*100:.1f}%)")
        
        # Pausa para não sobrecarregar a API
        time.sleep(0.5)
    
    print(f"Coleta de detalhes finalizada. Obtidos detalhes de {len(operadoras_detalhadas)} operadoras.")
    
    # 3. Converter para DataFrame e salvar
    if operadoras_detalhadas:
        df_detalhado = pd.DataFrame(operadoras_detalhadas)
        
        # Exibir informações
        print(f"\nDataFrame criado com sucesso. Dimensões: {df_detalhado.shape}")
        print("\nColunas disponíveis:")
        for col in df_detalhado.columns:
            print(f"- {col}")
        
        # Salvar CSV completo
        output_file = f"{output_dir}/ans_operadoras_detalhadas_{timestamp}.csv"
        df_detalhado.to_csv(output_file, index=False, encoding='utf-8')
        print(f"\nDados detalhados salvos em: {output_file}")
        
        # Salvar também em Excel para manter formatação
        excel_file = f"{output_dir}/ans_operadoras_detalhadas_{timestamp}.xlsx"
        df_detalhado.to_excel(excel_file, index=False)
        print(f"Dados detalhados também salvos em Excel: {excel_file}")
        
        # Informações adicionais
        if 'ativa' in df_detalhado.columns:
            ativas = df_detalhado['ativa'].sum() if df_detalhado['ativa'].dtype == 'bool' else df_detalhado['ativa'].astype(bool).sum()
            print(f"\nEstatísticas: {ativas} operadoras ativas e {len(df_detalhado) - ativas} inativas")
    else:
        print("Não foi possível obter dados detalhados das operadoras.")

# Executar o programa
if __name__ == "__main__":
    main()

Iniciando coleta da lista de operadoras em 2025-03-22 21:03:41...
Coletando página 1...
Informações de paginação: Página 1 de -1, Total de elementos: -1
Coletados 100 registros da página 1. Total até agora: 100
Coletando página 2...
Informações de paginação: Página 2 de -1, Total de elementos: -1
Coletados 100 registros da página 2. Total até agora: 200
Coletando página 3...
Informações de paginação: Página 3 de -1, Total de elementos: -1
Coletados 100 registros da página 3. Total até agora: 300
Coletando página 4...
Informações de paginação: Página 4 de -1, Total de elementos: -1
Coletados 100 registros da página 4. Total até agora: 400
Coletando página 5...
Informações de paginação: Página 5 de -1, Total de elementos: -1
Coletados 100 registros da página 5. Total até agora: 500
Coletando página 6...
Informações de paginação: Página 6 de -1, Total de elementos: -1
Coletados 100 registros da página 6. Total até agora: 600
Coletando página 7...
Informações de paginação: Página 7 de -1, 

Processados 50/4123 registros (1.2%)


Processados 100/4123 registros (2.4%)


Processados 150/4123 registros (3.6%)


Processados 200/4123 registros (4.9%)


...

Processados 4000/4123 registros (97.0%)


Processados 4050/4123 registros (98.2%)


Processados 4100/4123 registros (99.4%)


Coletando detalhes: 100%|██████████| 4123/4123 [42:37<00:00,  1.61it/s]


Coleta de detalhes finalizada. Obtidos detalhes de 4122 operadoras.

DataFrame criado com sucesso. Dimensões: (4122, 34)

Colunas disponíveis:
- registro_ans
- cnpj
- razao_social
- nome_fantasia
- ativa
- email
- site
- representante_nome
- representante_cargo
- autorizacao_funcionamento_em
- concessao_registro_definitivo_em
- registrada_em
- descredenciada_em
- descredenciamento_motivo
- classificacao_sigla
- classificacao_nome
- endereco_logradouro
- endereco_numero
- endereco_complemento
- endereco_bairro
- endereco_cep
- endereco_municipio_codigo
- endereco_municipio_nome
- endereco_uf_sigla
- endereco_valido
- telefone_ddd
- telefone_numero
- fax_ddd
- fax_numero
- link_self_href
- link_operadoras_href
- link_operadoras_templated
- segmentacao_sigla
- segmentacao_nome

Dados detalhados salvos em: /Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras/ans_operadoras_detalhadas_20250322_210341.csv
Dados detalhados também salvos em Excel: /Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras

In [9]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras/Relatorio_cadop_canceladas.csv', sep=";")

# Add new column 'Ativa' with value 'Cancelada' for all rows
df['Ativa'] = 'Cancelada'

# Save the modified dataframe back to CSV
df.to_csv('/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras/Relatorio_cadop_canceladas.csv', index=False)


In [10]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras/Relatorio_cadop.csv', sep=";")

# Add new column 'Ativa' with value 'Cancelada' for all rows
df['Ativa'] = 'Ativa'

# Save the modified dataframe back to CSV
df.to_csv('/Volumes/M1_SSD/DEV/ANS/Arquivos/dados/Operadoras/Relatorio_cadop.csv', index=False)


In [ ]:
junte os dois arqtuivos csv